# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")
data = data[data.country.isin(['Spain', 'Portugal', 'France', 'Italy', 'Belgium'])]

In [3]:
data = data.drop(['country','sex'], axis=1).groupby(['user'])['artist'].apply(lambda x: ';'.join(x)).reset_index()
data
#удаляем пол юзера, чтобы не строить закономерности на этом признаке

,user,artist
0,6,lily allen;kanye west;sigur rós;pink floyd;ste...
1,12,queen;the beatles;lynyrd skynyrd;neil young;th...
2,36,m.i.a.;fugazi;johnny cash;clint mansell;elton ...
3,47,pj harvey;radiohead;zero 7;einstürzende neubau...
4,74,in extremo;nine inch nails;billy talent;panic!...
...,...,...
1414,19690,pink floyd;the beatles;the white stripes;depec...
1415,19693,air;simon & garfunkel;pixies;serge gainsbourg;...
1416,19697,underworld;pendulum;the prodigy;sufjan stevens...
1417,19704,keane;pinback;coldplay;travis;calexico;doves;d...


In [4]:
binary_table = pd.concat([data.drop(['artist'], axis=1), data['artist'].str.get_dummies(';')], axis = 1).set_index(['user'])
binary_table
#преобразовываем данные к бинарному виду для использования Apriori и FPGrowth алгоритмов

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
user,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19690,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(binary_table, min_support=0.03, use_colnames=True,max_len = 5)\
                        .sort_values(by='support', ascending=False, na_position='last')
frequent_itemsets


,support,itemsets
122,0.237491,(radiohead)
33,0.188161,(coldplay)
141,0.174066,(the beatles)
99,0.174066,(muse)
113,0.130374,(pink floyd)
...,...,...
30,0.030303,(cansei de ser sexy)
253,0.030303,"(queen, the beatles)"
67,0.030303,(james blunt)
184,0.030303,"(radiohead, blur)"


In [6]:
#при поиске закономерностей будем устанавливать границы для consequent support и antecedent support, чтобы 
#находить зависимости между более популярными и менее популярными группами 
#Полезность наших закономерностей будет заключаться в продвижении менее популярных групп засчет 
# более популярных 
rules_confidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.02)
rules_confidence = rules_confidence[(rules_confidence['consequent support'] < 0.07) &
                                   (rules_confidence['antecedent support'] > 0.1)]\
.sort_values(by='confidence', ascending=False, na_position='last')
rules_confidence.head(1)
#находим закономерность, используя Априори и метрику Confidence

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
194,(the killers),(kaiser chiefs),0.109937,0.05074,0.031008,0.282051,5.558761,0.02543,1.322184


In [7]:
rules_confidence = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules_confidence = rules_confidence[(rules_confidence['consequent support'] < 0.1) &
                                   (rules_confidence['antecedent support'] > 0.1)]\
.sort_values(by='support', ascending=False, na_position='last')
rules_confidence.head(1)
#находим закономерность, используя Априори и метрику Support

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
18,(radiohead),(air),0.237491,0.092319,0.048626,0.204748,2.21784,0.026701,1.141376


In [8]:
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets = fpgrowth(binary_table, min_support=0.02, use_colnames=True, max_len=5)
frequent_itemsets.sort_values(by='support', ascending=False, na_position='last')
#Теперь вместо Априори используем FpGrowth

,support,itemsets
40,0.237491,(radiohead)
126,0.188161,(coldplay)
18,0.174066,(the beatles)
0,0.174066,(muse)
1,0.130374,(pink floyd)
...,...,...
628,0.020437,"(metallica, linkin park)"
308,0.020437,"(the beatles, u2)"
307,0.020437,"(the killers, u2)"
330,0.020437,"(radiohead, green day)"


In [9]:
# from mlxtend.frequent_patterns import fpmax
# frequent_itemsets = fpmax(binary_table, min_support=0.02, use_colnames=True)
# frequent_itemsets#.sort_values(by='support', ascending=False, na_position='last').head(10)
# fpmax не очень метрика нам не понравилось

In [10]:
rules_confidence = association_rules(frequent_itemsets, metric="conviction", min_threshold=1)
rules_confidence = rules_confidence[(rules_confidence['consequent support'] < 0.1) &
                                   (rules_confidence['antecedent support'] > 0.1)]\
.sort_values(by='conviction', ascending=False, na_position='last')
rules_confidence.head(1)
#fpgrowth алгоритм, используем метрику Conviction

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
728,(arctic monkeys),(franz ferdinand),0.10148,0.095842,0.03876,0.381944,3.985141,0.029034,1.462907


In [11]:
rules_confidence = association_rules(frequent_itemsets, metric="lift", min_threshold=0.01)
rules_confidence = rules_confidence[(rules_confidence['consequent support'] < 0.1) &
                                   (rules_confidence['antecedent support'] > 0.1)]\
.sort_values(by='lift', ascending=False, na_position='last')
rules_confidence.head(1)
#fpgrowth алгоритм, используем метрику Lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
373,(the killers),"(bloc party, the strokes)",0.109937,0.031712,0.021846,0.198718,6.266239,0.01836,1.208423


In [12]:
rules_confidence = association_rules(frequent_itemsets, metric="leverage", min_threshold=0.01)
rules_confidence = rules_confidence[(rules_confidence['consequent support'] < 0.1) &
                                   (rules_confidence['antecedent support'] > 0.1)]\
.sort_values(by='leverage', ascending=False, na_position='last')
rules_confidence.head(1)
#fpgrowth алгоритм, используем метрику Leverage

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
474,(coldplay),(oasis),0.188161,0.083157,0.047921,0.254682,3.062655,0.032274,1.230136
